# ML Model Horse Racing Predictions
#### Create an ML model that can predict whether a horse has a good chance of winning a race, based on the horse's features

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
cleaned_data_df = pd.read_csv("Data_Racing/International_Data/cleaned_data_df.csv")
cleaned_data_df.head()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-07-17 16:22:42.123733: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Unnamed: 0,ages,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,father,weight,res_win
0,1,4yo+,3218.0,10,6.0,7.0,0.444444,1,96.0,89.0,110.0,Getaway,65,0.0
1,26,5yo+,3218.0,5,8.0,1.0,0.266667,1,115.0,33.0,109.0,Beneficial,73,0.0
2,27,5yo+,3218.0,5,9.0,2.0,0.125000,0,108.0,29.0,103.0,Beneficial,68,0.0
3,43,4yo+,4827.0,6,7.0,13.0,0.076923,0,95.0,39.0,83.0,Kalanisi,63,1.0
4,44,4yo+,4827.0,6,8.0,2.0,0.076923,0,119.0,64.0,108.0,Ask,71,0.0


In [2]:
# Determine the number of unique values in each column.
cleaned_data_df.nunique()

Unnamed: 0      24922
ages               16
metric             52
ncond              13
age                14
saddle             29
decimalPrice       79
isFav               2
RPR               165
TR                157
OR                149
father            327
weight             32
res_win             2
dtype: int64

## Binning (not used for now)

In [ ]:
# Look at value counts for binning
res_win_counts = cleaned_data_df["res_win"].value_counts()
res_win_counts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
res_win_to_replace = list(res_win_counts[winning_time_counts < 900].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts.loc[classification_counts > 1]

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 800].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

## Post Binning

In [3]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_data = pd.get_dummies(cleaned_data_df)
converted_data.head()

,Unnamed: 0,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,...,father_Well Chosen,father_Westerner,father_Whipper,father_Winged Love,father_Wootton Bassett,father_Yeats,father_Youmzain,father_Zanzibari,father_Zebedee,father_Zoffany
0,1,3218.0,10,6.0,7.0,0.444444,1,96.0,89.0,110.0,...,0,0,0,0,0,0,0,0,0,0
1,26,3218.0,5,8.0,1.0,0.266667,1,115.0,33.0,109.0,...,0,0,0,0,0,0,0,0,0,0
2,27,3218.0,5,9.0,2.0,0.125000,0,108.0,29.0,103.0,...,0,0,0,0,0,0,0,0,0,0
3,43,4827.0,6,7.0,13.0,0.076923,0,95.0,39.0,83.0,...,0,0,0,0,0,0,0,0,0,0
4,44,4827.0,6,8.0,2.0,0.076923,0,119.0,64.0,108.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Split our preprocessed data into our features and target arrays
y = converted_data['res_win']
X = converted_data.drop(columns=['res_win'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
input_features = len(X_train_scaled[0])
input_features

354

## Compile, Train and Evaluate the Model

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
neurons_hidden_layer1 = 256
neurons_hidden_layer2 = 148
neurons_hidden_layer3 = 76

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer1, activation="relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               90880     
                                                                 
 dense_5 (Dense)             (None, 148)               38036     
                                                                 
 dense_6 (Dense)             (None, 76)                11324     
                                                                 
 dense_7 (Dense)             (None, 1)                 77        
                                                                 
Total params: 140,317
Trainable params: 140,317
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 250)

Epoch 1/250
585/585 [==============================] - 1s 939us/step - loss: 0.3416 - accuracy: 0.8733
Epoch 2/250
585/585 [==============================] - 1s 925us/step - loss: 0.2914 - accuracy: 0.8820
Epoch 3/250
585/585 [==============================] - 1s 921us/step - loss: 0.2676 - accuracy: 0.8878
Epoch 4/250
585/585 [==============================] - 1s 922us/step - loss: 0.2459 - accuracy: 0.8918
Epoch 5/250
585/585 [==============================] - 1s 920us/step - loss: 0.2300 - accuracy: 0.8964
Epoch 6/250
585/585 [==============================] - 1s 919us/step - loss: 0.2154 - accuracy: 0.9047
Epoch 7/250
585/585 [==============================] - 1s 917us/step - loss: 0.2058 - accuracy: 0.9069
Epoch 8/250
585/585 [==============================] - 1s 1ms/step - loss: 0.1929 - accuracy: 0.9145
Epoch 9/250
585/585 [==============================] - 1s 1ms/step - loss: 0.1856 - accuracy: 0.9180
Epoch 10/250
585/585 [==============================] - 1s 940us/step - loss:

585/585 [==============================] - 1s 971us/step - loss: 0.0123 - accuracy: 0.9956
Epoch 81/250
585/585 [==============================] - 1s 1ms/step - loss: 0.0165 - accuracy: 0.9951
Epoch 82/250
585/585 [==============================] - 1s 950us/step - loss: 0.0167 - accuracy: 0.9946
Epoch 83/250
585/585 [==============================] - 1s 952us/step - loss: 0.0139 - accuracy: 0.9952
Epoch 84/250
585/585 [==============================] - 1s 942us/step - loss: 0.0188 - accuracy: 0.9941
Epoch 85/250
585/585 [==============================] - 1s 954us/step - loss: 0.0194 - accuracy: 0.9940
Epoch 86/250
585/585 [==============================] - 1s 949us/step - loss: 0.0143 - accuracy: 0.9949
Epoch 87/250
585/585 [==============================] - 1s 961us/step - loss: 0.0136 - accuracy: 0.9953
Epoch 88/250
585/585 [==============================] - 1s 957us/step - loss: 0.0144 - accuracy: 0.9951
Epoch 89/250
585/585 [==============================] - 1s 953us/step - loss: 0

585/585 [==============================] - 1s 931us/step - loss: 0.0072 - accuracy: 0.9981
Epoch 159/250
585/585 [==============================] - 1s 922us/step - loss: 0.0051 - accuracy: 0.9984
Epoch 160/250
585/585 [==============================] - 1s 918us/step - loss: 0.0099 - accuracy: 0.9973
Epoch 161/250
585/585 [==============================] - 1s 923us/step - loss: 0.0077 - accuracy: 0.9977
Epoch 162/250
585/585 [==============================] - 1s 923us/step - loss: 0.0099 - accuracy: 0.9967
Epoch 163/250
585/585 [==============================] - 1s 922us/step - loss: 0.0056 - accuracy: 0.9980
Epoch 164/250
585/585 [==============================] - 1s 936us/step - loss: 0.0046 - accuracy: 0.9987
Epoch 165/250
585/585 [==============================] - 1s 931us/step - loss: 0.0025 - accuracy: 0.9990
Epoch 166/250
585/585 [==============================] - 1s 924us/step - loss: 0.0027 - accuracy: 0.9991
Epoch 167/250
585/585 [==============================] - 1s 917us/ste

585/585 [==============================] - 1s 1ms/step - loss: 0.0032 - accuracy: 0.9987
Epoch 237/250
585/585 [==============================] - 1s 938us/step - loss: 0.0040 - accuracy: 0.9987
Epoch 238/250
585/585 [==============================] - 1s 953us/step - loss: 0.0060 - accuracy: 0.9983
Epoch 239/250
585/585 [==============================] - 1s 954us/step - loss: 0.0098 - accuracy: 0.9976
Epoch 240/250
585/585 [==============================] - 1s 959us/step - loss: 0.0052 - accuracy: 0.9983
Epoch 241/250
585/585 [==============================] - 1s 952us/step - loss: 0.0041 - accuracy: 0.9987
Epoch 242/250
585/585 [==============================] - 1s 953us/step - loss: 0.0046 - accuracy: 0.9990
Epoch 243/250
585/585 [==============================] - 1s 959us/step - loss: 0.0034 - accuracy: 0.9987
Epoch 244/250
585/585 [==============================] - 1s 962us/step - loss: 0.0043 - accuracy: 0.9991
Epoch 245/250
585/585 [==============================] - 1s 953us/step 

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

195/195 - 0s - loss: 1.7877 - accuracy: 0.8787 - 299ms/epoch - 2ms/step
Loss: 1.7877320051193237, Accuracy: 0.8786711692810059


In [29]:
(y_test == 0).mean()

0.8693628631038357

In [18]:
# Make predictions on the test data
y_pred = nn.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)

195/195 [==============================] - 0s 478us/step


In [19]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_binary)
print("Precision:", precision)

Precision: 0.5516014234875445


In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

         0.0       0.91      0.95      0.93      5417
         1.0       0.55      0.38      0.45       814

    accuracy                           0.88      6231
   macro avg       0.73      0.67      0.69      6231
weighted avg       0.86      0.88      0.87      6231



In [ ]:
# Export our model to HDF5 file
# nn.save('h5_files/AlphabetSoup_Model4.h5')